# Semantic role labeling with BERT

In this notebook, you'll perform semantic role labeling with BERT, using the [English Universal Propbank 1.0 datasets](https://github.com/UniversalPropositions/UP-1.0).

## Before you begin

### Install libraries

If you run this notebook in [Google colab](https://colab.research.google.com), make sure to install the required packages:

In [27]:
try:
    import google.colab
    runs_in_colab =  True
except ImportError:
    runs_in_colab = False

if runs_in_colab:
    !pip install datasets
    !pip install accelerate==0.21.0
    !pip install transformers[torch]
    !pip install accelerate -U

    # Import the drive library to save your model, tokenizer and trainer to Google Drive
    from google.colab import drive
    drive.mount('/content/drive/')

### Import libraries

In [30]:
import time
import pandas as pd
import transformers
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
from utils import read_data_as_sentence,map_labels_in_dataframe,tokenize_and_align_labels,get_label_mapping,get_labels_from_map,load_srl_model,load_dataset,compute_metrics,write_predictions_to_csv,compute_evaluation_metrics_from_csv, print_sentences
from sklearn.metrics import precision_recall_fscore_support
from main_bert_srl import main, define_args

## Step 1: Preprocess data

Unlike traditional token labeling methods, which assign labels to individual words in isolation, BERT performs sequence labeling. This means BERT assigns labels to individual tokens, while taking the full sentence context in consideration.

The English Universal PropBank 1.0 dataset is structured in [CoNNL-U Plus format](https://universaldependencies.org/ext-format.html), in which lines represent individual tokens. So before you can train the model, you need to extract sentences and labels from the datasets, and preprocess the sentences by removing non-argument labels.

To preprocess the datasets and save the resulting DataFrame to a file, call the `read_data_as_sentence()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1*  (string)                 | ✅️ | The filepath for the CoNNLU dataset. |
| *positional 2*  (string)               | ✅ | The filepath to write the preprocessed DataFrame to. |
| `mode` (enum)                          | Optional (defaults to **basic**) | The method of preprocessing, used for the advanced model. |

In [31]:
train_data = read_data_as_sentence('data/en_ewt-up-train.conllu', 'data/en_ewt-up-train.preprocessed.csv')
dev_data = read_data_as_sentence('data/en_ewt-up-dev.conllu', 'data/en_ewt-up-dev.preprocessed.csv')
test_data = read_data_as_sentence('data/en_ewt-up-test.conllu', 'data/en_ewt-up-test.preprocessed.csv')

The `read_data_as_sentence()` function returns DataFrames, where each row represents a sentence from the dataset passed to the function. Each sentence has been expanded based on its predicates, resulting in multiple copies of the same sentence, each focused on a different predicate.

The DataFrame has two columns:

- `input_form`: a list of strings, where each string represents a words in the sentence, followed by two special tokens:
    1. A special token (`[SEP]`), which denotes the separation between the words of the sentence and the predicate form.
    2. The predicate form, which corresponds to the `argument` values for the same row in the DataFrame.
- `argument`: a list of strings, representing the arguments associated with each word in the sentence. The length of each list is equal to the number of words in the sentence, plus two additional elements, for the special token and predicate form. The arguments match the predicate appended to the `input_form` for the same row in the DataFrame.

### Explore the DataFrame

Before you continue to tokenize the sentences and fine-tune the BERT model, it's time to get more familiar with our data.

To explore the DataFrame, start by printing the head of the preprocessed DataFrame:

In [32]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3971 entries, 0 to 3970
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_form  3971 non-null   object
 1   argument    3971 non-null   object
dtypes: object(2)
memory usage: 62.2+ KB
None


The **Non-Null** count for both columns should match, indicating there are as many lists of `input_form` values as there are lists of `argument` values, namely one for each sentence.

Next, print the words and their argument labels for the first 20 sentences of the test dataset:

In [33]:
print_sentences(test_data)

form: What            argument: _
form: if              argument: _
form: Google          argument: ARG1
form: Morphed         argument: _
form: Into            argument: _
form: GoogleOS        argument: ARG2
form: ?               argument: _
----------------------------------------
form: [SEP]           argument: None
form: Morphed         argument: None


form: What            argument: _
form: if              argument: _
form: Google          argument: ARG0
form: expanded        argument: _
form: on              argument: _
form: its             argument: _
form: search          argument: _
form: -               argument: _
form: engine          argument: _
form: (               argument: _
form: and             argument: _
form: now             argument: _
form: e-mail          argument: _
form: )               argument: _
form: wares           argument: ARG1
form: into            argument: _
form: a               argument: _
form: full            argument: _
form: -              

As you can see, the sequence of word forms runs parallel to the sequence of argument labels. This means that for every index of `input_form`, the same index of `argument` gives its argument label.

Argument labels are:
- **'_'** for tokens that are not an argument (in the current predicate sense of the sentence).
- The token's respective Propbank label for tokens that are an argument, e.g. **ARG1**
- **None** for the special separator token (`[SEP]`) and the predicate token that follows the separator.

For example, in the the first sentence of the test data printed above ("What if Google Morphed Into GoogleOS?"), the predicate 'Morphed' evokes [the frame `morph.01`](https://propbank.github.io/v3.4.0/frames/morph.html#morph.01). The frame's arguments are:

- `ARG0-PAG`: causer of transformation
- `ARG1-PPT`: thing changing
- `ARG2-PRD`: end state
- `ARG3-VSP`: start state

In this example, the `ARG1` label is assigned to 'Google', and the `ARG2` label is assigned to 'GoogleOS', which indicates 'Google' is the thing that is changing and 'GoogleOS' is its end state.

## Step 2: Initialize a tokenizer

Now that you have extracted sentences and labels from the datasets, you need to prepare the sentences for the BERT model by tokenizing them.

Use HuggingFace's [`AutoTokenizer`](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/auto#transformers.AutoTokenizer) to construct a DistilBERT tokenizer, which is based on the WordPiece algorithm.

In [34]:
# Set the model ID to use
model_id = "distilbert-base-uncased"

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Check the assertion that the tokenizer is an instance of transformers.PreTrainedTokenizerFast
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/kris/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.0",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/kris/.cache/huggingface/transformers/0e1bbfda7f63a99bb

To test the `tokenizer()`, tokenize the first sentence of the test data, including:

- `add_special_tokens` set to **True** to add a `[CLS]` token to the start of every sentence.
- `is_split_into_words` set to **True** because the sentence is already split into words (based on the Universal Propbank 1.0 dataset)

In [35]:
# Tokenize the first example in the test data
example = test_data['input_form'][0]
tokenized_input = tokenizer(example,add_special_tokens=True, is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

# Print the example tokens and their corresponding IDs
for token, id in zip(tokens, tokenized_input["input_ids"]):
    print(f"{token:>10} {id}")

     [CLS] 101
      what 2054
        if 2065
    google 8224
       mor 22822
      ##ph 8458
      ##ed 2098
      into 2046
    google 8224
      ##os 2891
         ? 1029
     [SEP] 102
       mor 22822
      ##ph 8458
      ##ed 2098
     [SEP] 102


You've successfully tokenized the sample sentence, splitting words up into subword tokens and fetching their token IDs from DistilBERT's vocabulary.

> Note: notice how the special tokens `[CLS]` and `[SEP]` are tokenized as **101** and **102**. These numbers are meaningful to BERT.

## Step 3: Prepare the input for training

Before training the model, map the labels in the datasets to numerical values. This ensures consistency and facilitates the training process.

To get the label mapping, call `get_label_mapping()`, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (DataFrame)          | ✅️ | The training dataset for which to extract the label mapping. |
| *positional 2* (DataFrame)          | ✅ | The test dataset for which to extract the label mapping.|
| *positional 3* (DataFrame)          | ✅ | The dev dataset for which to extract the label mapping. |

In [36]:
label_map = get_label_mapping(train_data, test_data, dev_data)

The `get_label_mapping()` function returns an alphabetically-ordered dictionary mapping:
- **_** to **0**.
- String labels to integers, e.g. **ARG0** to **1**.
- **None** to **None**, to preserve the labels for special tokens and predicates. (You will replace **None** with **-100** later to mask these tokens from being labeled.)

In [37]:
print(label_map)

{'_': 0, 'ARG0': 1, 'ARG1': 2, 'ARG1-DSP': 3, 'ARG2': 4, 'ARG3': 5, 'ARG4': 6, 'ARG5': 7, 'ARGA': 8, 'ARGM-ADJ': 9, 'ARGM-ADV': 10, 'ARGM-CAU': 11, 'ARGM-COM': 12, 'ARGM-CXN': 13, 'ARGM-DIR': 14, 'ARGM-DIS': 15, 'ARGM-EXT': 16, 'ARGM-GOL': 17, 'ARGM-LOC': 18, 'ARGM-LVB': 19, 'ARGM-MNR': 20, 'ARGM-MOD': 21, 'ARGM-NEG': 22, 'ARGM-PRD': 23, 'ARGM-PRP': 24, 'ARGM-PRR': 25, 'ARGM-REC': 26, 'ARGM-TMP': 27, 'C-ARG0': 28, 'C-ARG1': 29, 'C-ARG1-DSP': 30, 'C-ARG2': 31, 'C-ARG3': 32, 'C-ARG4': 33, 'C-ARGM-ADV': 34, 'C-ARGM-COM': 35, 'C-ARGM-CXN': 36, 'C-ARGM-DIR': 37, 'C-ARGM-EXT': 38, 'C-ARGM-GOL': 39, 'C-ARGM-LOC': 40, 'C-ARGM-MNR': 41, 'C-ARGM-PRP': 42, 'C-ARGM-PRR': 43, 'C-ARGM-TMP': 44, 'R-ARG0': 45, 'R-ARG1': 46, 'R-ARG2': 47, 'R-ARG3': 48, 'R-ARG4': 49, 'R-ARGM-ADJ': 50, 'R-ARGM-ADV': 51, 'R-ARGM-CAU': 52, 'R-ARGM-COM': 53, 'R-ARGM-DIR': 54, 'R-ARGM-GOL': 55, 'R-ARGM-LOC': 56, 'R-ARGM-MNR': 57, 'R-ARGM-TMP': 58, None: None}



Next, apply the label mapping to the datasets, adding the column `mapped_labels` to the DataFrames. This column contains arrays of integers representing the labels, based on the label mapping.

To apply the label mapping, call `map_labels_in_dataframe()`, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1*                   | ✅️ | The DataFrame for which to convert the argument labels. |
| *positional 2*                 | ✅ | The label mapping, created with `get_label_mapping()`. |

In [38]:
train_data = map_labels_in_dataframe(train_data, label_map)
dev_data = map_labels_in_dataframe(dev_data, label_map)
test_data = map_labels_in_dataframe(test_data, label_map)

As you can see, for each row in the DataFrame, the values in `mapped_labels` and `arguments` correspond to the mapping in `label_map`:

In [39]:
test_data.head()

,input_form,argument,mapped_labels
0,"[What, if, Google, Morphed, Into, GoogleOS, ?,...","[_, _, ARG1, _, _, ARG2, _, None, None]","[0, 0, 2, 0, 0, 4, 0, None, None]"
1,"[What, if, Google, expanded, on, its, search, ...","[_, _, ARG0, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, ..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, ARGM-DIS, _, _, ARG1, _, _, _,...","[0, 0, 0, 0, 0, 15, 0, 0, 2, 0, 0, 0, 0, 4, 0,..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[This, BuzzMachine, post, argues, that, Google...","[_, ARG2, _, _, _, _, _, _, _, _, _, _, _, _, ...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Now that you have initialized and tested the `tokenizer()` and added mapped labels to the DataFrames, it's time to tokenize (and pad) all sentences.

Since WordPiece tokenization potentially breaks words up into subword tokens, the tokens and their labels have to be re-aligned. The `tokenize_and_align_labels()` function you'll call for this iterates over each token and determines the appropriate label based on the provided dataset.

Special tokens are assigned a label of **-100** to indicate they should be ignored in the loss function. Labels for the first token of each word are set accordingly, while labels for subsequent tokens within the same word are determined based on the `label_all_tokens` flag.

To tokenize the sentences and align the labels, call `tokenize_and_align_labels()`, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (`transformers AutoTokenizer`) | ✅️ | The `tokenizer()` for the pre-trained model. |
| *positional 2* (DataFrame) | ✅ | The preprocessed datasets |
| `label_all_tokens` (boolean)     | Optional (defaults to **True**) | Whether all tokens should receive their own label, accounting for words split into subtokens |

In [40]:
tokenized_test = tokenize_and_align_labels(tokenizer, test_data, label_all_tokens=True)
tokenized_train = tokenize_and_align_labels(tokenizer, train_data, label_all_tokens=True)
tokenized_dev = tokenize_and_align_labels(tokenizer, dev_data, label_all_tokens=True)

Now that you have tokenized all three datasets, let's examine the result.

The `tokenized_` datasets are of the type `transformers.tokenization_utils_base.BatchEncoding` and have three attributes per row:

1. `input_ids`: an array of token IDs for the tokenized sentence. Starts with the token ID for the `[CLS]` token, followed by the tokenized sentence, the `[SEP]` token, the predicate, and a final `[SEP]` token.
2. `attention_mask`: an array representing the attention mask for the sentence.
3. `labels`: an array with numerical labels, aligned with the tokens.

> Note: all three arrays are padded so that every sample per dataset is of equal length.


In [41]:
print(type(tokenized_test))
print(tokenized_test.keys())
print(tokenizer.convert_ids_to_tokens(tokenized_test["input_ids"][0]))
for key in tokenized_test.keys():
    print(f"{key}: {tokenized_test[key][0]}")

<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask', 'labels'])
['[CLS]', 'what', 'if', 'google', 'mor', '##ph', '##ed', 'into', 'google', '##os', '?', '[SEP]', 'mor', '##ph', '##ed', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
input_ids: tensor([  101,  2

To confirm that you have padded all sentences in the `tokenized_test` dataset to be of equal length, let's check the length of all three arrays for the first 10 sentences:

In [43]:
for i in range(10):
    print(f"sentence {i}:", "input_ids:", len(tokenized_test["input_ids"][i]), "\tlabels:", len(tokenized_test["labels"][i]), "\tattention_mask:", len(tokenized_test["attention_mask"][i]))

sentence 0: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 1: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 2: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 3: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 4: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 5: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 6: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 7: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 8: input_ids: 97 	labels: 97 	attention_mask: 97
sentence 9: input_ids: 97 	labels: 97 	attention_mask: 97


Now that you have tokenized and padded the sentences, and aligned the labels with the tokens, you're ready to transform the tokenized datasets into Hugging Face's [`datasets.arrow_dataset.Dataset`](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset).

To transform the tokenized datasets into `Dataset` objects, call the `load_dataset()` function, which calls the [`Dataset.from_dict()` method](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.from_dict), including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (`transformers.tokenization_utils_base.BatchEncoding`) | ✅️ | The tokenized dataset. |

In [44]:
dataset_train = load_dataset(tokenized_train)
dataset_dev = load_dataset(tokenized_dev)
dataset_test = load_dataset(tokenized_test)

Let's print the type of the resulting dataset, to confirm the transformation into `datasets.arrow_dataset.Dataset`:

In [45]:
print(type(dataset_test))

<class 'datasets.arrow_dataset.Dataset'>


## Step 4: Fine-tune the model

Finally, the sentences have been transformed from CoNNL-U Plus format to Hugging Face `Dataset` objects: it's time to fine-tune BERT!

Fine-tuning a BERT model on the full dataset can be a very computationally challenging task. To speed up the process, create subsets of the three datasets with 1000 samples per dataset, selected randomly:

In [46]:
small_train_dataset = dataset_train.shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset_dev.shuffle(seed=42).select(range(1000))
small_test_dataset = dataset_test.shuffle(seed=42).select(range(1000))

To map the numerical labels back to their string representations, you need to convert the `label_map` dictionary to a list of labels (as strings).

To convert the `label_map` to a list of labels (as strings), call the `get_labels_from_map()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (dictionary) | ✅️ | The dictionary mapping labels as strings to their numerical represenation. |

In [47]:
label_list = get_labels_from_map(label_map)

Next, load the pretrained DistilBERT model using the [`AutoModelForTokenClassification.from_pretrained()` method](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.FlaxAutoModelForVision2Seq.from_pretrained) from the `transformers` library, together with the model name (**distilbert-base-uncased**), and the [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/trainer#transformers.TrainingArguments) neccesary for training.

To get the model, model name and `TrainingArguments`, call the `load_srl_model()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (string) | ✅️ | The model identifier.  |
| *positional 2* (list of strings) | ✅️ | The tokenized dataset. |
| `batch_size` (integer) | Optional (defaults to **16**) | The [batch size for training](https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#methods-and-tools-for-efficient-training-on-a-single-gpu) and [inference](https://huggingface.co/docs/setfit/main/en/how_to/batch_sizes). |

In [48]:
model, args = load_srl_model(model_id, label_list)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/kris/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_

Now that you have a DistilBERT model, it's time for fine-tuning the model for the task of semantic role labeling (SRL).

To fine-tune your model, instantiate a [`Trainer` object](https://huggingface.co/docs/transformers/main_classes/trainer#api-reference%20][%20transformers.Trainer) from the `transformers` library, passing the `model`, `args`, `tokenizer` and datasets for training and inference. Then, call the [`Trainer.train()` method](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.train) to start the fine-tuning process.

> Note: this process may take up to several hours, depending on your hardware.

In [144]:
trainer = Trainer(
        model,
        args,
        train_dataset=small_train_dataset,
        eval_dataset=dataset_dev,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(*p, label_list, tokenized_dev)
    )
trainer.train()

/Users/kris/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 189


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.418899,0.100751,0.082640,0.084789,0.911883
2,No log,0.388719,0.139374,0.091278,0.093028,0.918254
3,No log,0.380618,0.138709,0.094296,0.097516,0.919700


***** Running Evaluation *****
  Num examples = 4143
  Batch size = 16
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

TrainOutput(global_step=189, training_loss=0.2894454834953187, metrics={'train_runtime': 4807.0499, 'train_samples_per_second': 0.624, 'train_steps_per_second': 0.039, 'total_flos': 138706544862000.0, 'train_loss': 0.2894454834953187, 'epoch': 3.0})

Now that you have fine-tuned the model, let's evaluate its performance on the `eval_dataset` that you set when constructing the `Trainer` instance.

To evaluate the fine-tuned model, call the [`Trainer.evaluate()` method](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.evaluate).

In [145]:
metrics = trainer.evaluate()
print(metrics)

***** Running Evaluation *****
  Num examples = 4143
  Batch size = 16


KeyboardInterrupt: 

Now that you have fine-tuned your DistilBERT model for semantic role labeling, and evaluated its performance on the development dataset, it's time to infer the argument labels of the test dataset and compute a summary of the performance metrics.

First, call the [`Trainer.predict()` method](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.predict) passing the test dataset. The method returns a tuple consisting of the model's predictions on the test dataset, the labels, and metrics.

To compute a summary of the model's perfomance metrics on the test dataset, call the `compute_metrics()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (`np.ndarray`) | ✅️ | The array of predictions as returned from the `Trainer.predict()` method. |
| *positional 2* (`np.ndarray`) | ✅️ | The array of argument labels as returned from the `Trainer.predict()` method. |
| *positional 3* (list of strings) | ✅️ | The list of argument labels as strings. |

In [146]:
predictions, labels, _ = trainer.predict(dataset_dev)

***** Running Prediction *****
  Num examples = 4143
  Batch size = 16
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [263]:
from sklearn.metrics import classification_report

def compute_metrics(predictions, labels, label_list, tokenized_dataset, tokenizer, dev_data):
    """
    Compute evaluation metrics for Semantic Role Labeling (SRL).
    Return a dictionary with evaluation metrics.
    """
    # The prediction with the highest probability is the predicted label.
    predictions_argmax = np.argmax(predictions, axis=2)

    # Create empty lists for the predictions and true labels.
    predictions = []
    true_labels = []

    # Iterate over the predictions and true labels, every iteration is one sentence.
    for sentence_nr, (prediction, label) in enumerate(zip(predictions_argmax, labels)):

        filtered_prediction = []
        filtered_label = []

        # Get the token-to-word mapping, tokens and sentence for the current sentence.
        token_to_word_mapping = tokenized_dataset.word_ids(sentence_nr)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_dataset["input_ids"][sentence_nr])
        sentence = dev_data.input_form[sentence_nr]
    
        # Iterate over the predictions and true labels, every iteration is one token.
        for token_nr, (p, l, t2w_map, t) in enumerate(zip(prediction, label, token_to_word_mapping, tokens)):

            # Find the word that the current token belongs to.
            word = sentence[t2w_map] if t2w_map != None else ""

            # Print the prediction, label, token-to-word mapping, token and word.
            # print(f"p: {p}, l: {l}, t2w_map: {t2w_map}, t: {t}, w: {word}")
            
            # Skip the first [CLS] token.
            if t == "[CLS]":
                continue

            # If the token is the [SEP] token, break the loop.
            if t == "[SEP]":
                break

            # Check if this token belongs to the same word as the previous token.
            if t2w_map == token_to_word_mapping[token_nr - 1]:
                print("same as last! {t2w_map} {token_to_word_mapping[token_nr - 1]}")
                continue
            
            # Append the prediction and label to the filtered lists.
            filtered_prediction.append(label_list[p])
            filtered_label.append(label_list[l])
    
        predictions.append(filtered_prediction)
        true_labels.append(filtered_label)

    # Flatten the predictions and true labels to compute the metrics.
    predict = [label for sentence in predictions for label in sentence]
    gold = [label for sentence in true_labels for label in sentence]

    print(len(predict))
    print(len(gold))

    # Create a list of all possible labels from gold and inferenced labels.
    labels=list(set(predict+gold))

    report = classification_report(gold,predict,target_names=labels,output_dict=True)

    # Print the classification report.
    print(classification_report(gold,predict))

    return {
           "precision": report["macro avg"]["precision"],
    "recall": report["macro avg"]["recall"],
    "f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"]
    }

In [264]:
print(dev_data.input_form[0])

['From', 'the', 'AP', 'comes', 'this', 'story', ':', '[SEP]', 'comes']


In [265]:
results = compute_metrics(predictions, labels, label_list, tokenized_dev, tokenizer, dev_data)
print(results)

87697
87697


/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kris/anaconda3/lib/pytho

              precision    recall  f1-score   support

        ARG0       0.65      0.69      0.67      1733
        ARG1       0.52      0.65      0.58      3325
        ARG2       0.62      0.22      0.32      1212
        ARG3       0.00      0.00      0.00        77
        ARG4       0.00      0.00      0.00        47
        ARG5       0.00      0.00      0.00         1
    ARGM-ADJ       0.00      0.00      0.00       251
    ARGM-ADV       0.20      0.00      0.01       481
    ARGM-CAU       0.00      0.00      0.00        70
    ARGM-COM       0.00      0.00      0.00        14
    ARGM-CXN       0.00      0.00      0.00        14
    ARGM-DIR       0.00      0.00      0.00        48
    ARGM-DIS       0.80      0.02      0.04       186
    ARGM-EXT       0.00      0.00      0.00       113
    ARGM-GOL       0.00      0.00      0.00        26
    ARGM-LOC       0.00      0.00      0.00       242
    ARGM-LVB       0.00      0.00      0.00        75
    ARGM-MNR       0.00    

/Users/kris/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Writing the predictions together with the gold labels to a csv file with the function `write_predictions_to_csv` so that the metrics per class can be computed with the `compute_evaluation_metrics_from_csv` function.

Now that you have succesfully fine-tuned a model and inferred the argument labels of the test set, let's store the results on disc in a CSV file, and create a full classification report.

To write the predictions to CSV, call the `write_predictions_to_csv()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (`np.ndarray`) | ✅️ | The array of predictions as returned from the `Trainer.predict()` method. |
| *positional 2* (`np.ndarray`) | ✅️ | The array of argument labels as returned from the `Trainer.predict()` method. |
| *positional 3* (list of strings) | ✅️ | The list of argument labels as strings. |
| *positional 3* (string) | ✅️ | The filepath to write the results to (in CSV format). |

Next, to compute a full classification report of the model's performance on the test dataset, call the `compute_evaluation_metrics_from_csv()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1* (string) | ✅️ | The filepath for the CSV file where the model's predictions are stored. |

In [23]:
results_file = "predictions.csv"
write_predictions_to_csv(predictions, labels, label_list, results_file)
classification_report = compute_evaluation_metrics_from_csv("predictions.csv")
print(classification_report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        ARG0       0.78      0.82      0.80      1899
        ARG1       0.76      0.82      0.79      3647
    ARG1-DSP       0.00      0.00      0.00         4
        ARG2       0.62      0.61      0.62      1285
        ARG3       0.00      0.00      0.00        79
        ARG4       1.00      0.09      0.16        68
        ARG5       0.00      0.00      0.00         1
        ARGA       0.00      0.00      0.00         2
    ARGM-ADJ       0.66      0.69      0.67       254
    ARGM-ADV       0.67      0.39      0.49       526
    ARGM-CAU       0.00      0.00      0.00        48
    ARGM-COM       0.00      0.00      0.00        16
    ARGM-CXN       0.00      0.00      0.00        12
    ARGM-DIR       0.33      0.11      0.16        47
    ARGM-DIS       0.69      0.59      0.64       196
    ARGM-EXT       0.78      0.70      0.74       105
    ARGM-GOL       0.00      0.00      0.00        29
    ARGM-LOC       0.52    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finally, store the `tokenizer`, `trainer` and `model` on disc using their built-in methods. For each method call, pass a string representing the directory to save the object and its configuration to.

This let's you use the objects' built-in `from_pretrained()` methods to reload their state.

In [24]:
# Use these codes to save model:
tokenizer.save_pretrained("tokenizer.save_pretrained.distillbert-base-uncased-finetuned-srl")
trainer.save_model("trainer.save_model.distillbert-base-uncased-finetuned-srl")
model.save_pretrained("model.save_pretrained.distillbert-base-uncased-finetuned-srl")

If you are running this notebook in Google colab, create a directory in your Google Drive and copy the `tokenizer`, `trainer`, and `model` to your Google Drive:

In [25]:
if runs_in_colab:
    !mkdir -p "/content/drive/MyDrive/NLP_3_baseline_model/model"
    !cp -r '/content/trainer.save_model.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'
    !cp -r '/content/model.save_pretrained.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'
    !cp -r '/content/tokenizer.save_pretrained.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'

In [26]:
classification_report = compute_evaluation_metrics_from_csv("predictions-adv.csv")
print(classification_report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        ARG0       0.79      0.82      0.80      1899
        ARG1       0.75      0.83      0.79      3647
    ARG1-DSP       0.00      0.00      0.00         4
        ARG2       0.60      0.65      0.62      1285
        ARG3       0.00      0.00      0.00        79
        ARG4       0.00      0.00      0.00        68
        ARG5       0.00      0.00      0.00         1
        ARGA       0.00      0.00      0.00         2
    ARGM-ADJ       0.68      0.68      0.68       254
    ARGM-ADV       0.62      0.41      0.49       526
    ARGM-CAU       1.00      0.02      0.04        48
    ARGM-COM       0.00      0.00      0.00        16
    ARGM-CXN       0.00      0.00      0.00        12
    ARGM-DIR       0.27      0.15      0.19        47
    ARGM-DIS       0.70      0.56      0.62       196
    ARGM-EXT       0.74      0.69      0.71       105
    ARGM-GOL       0.00      0.00      0.00        29
    ARGM-LOC       0.54    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Limitations of the basic model

In the basic model, input sequences start with the token ID for the `[CLS]` token, followed by the tokenized sentence, the `[SEP]` token, the predicate, and a final `[SEP]` token. However, this method introduces ambiguity when there are multiple word forms in a sentence that are identical to the predicate word form, for example:

```
['[CLS]', 'she', 'saw', 'a', 'man', 'saw', 'a', 'tree', '[SEP]', 'saw', '[SEP]']
```

In the sentence above, the predicate word form 'saw' occurs twice in the sentence, with different frames and arguments for each occurence:
1. In the first occurence, the frame evoked is that of [`see.01` *(view)*](https://propbank.github.io/v3.4.0/frames/see.html#see.01), which has the following arguments:

    - `ARG0-PAG`: viewer
    - `ARG1-PPT`: thing viewed
    - `ARG2-PRD`: attribute of arg1, further description

  - In this frame, 'she' is labeled `ARG0` and 'man' is labeled `ARG1`.

2. In the second occurence, the frame evoked is that of [`saw.06` *(cut with a saw)*](https://propbank.github.io/v3.4.0/frames/see.html#saw.06), which has the following arguments:

    - `ARG0-PAG`: cutter
    - `ARG1-PPT`: thing cut
    - `ARG2-SRC`: Cut from what? source 

  - In this frame, 'man' is labeled `ARG0` and 'tree' is labeled `ARG1`.

Due to limitations in how your basic model represents these input sequences, the model isn't able to disambiguate between the two frames in the sentence above, introducing noise in the model. 

## Advanced model

To resolve the limitation of the basic model, you're going to create an advanced model, using a more sophisticated representation of the input sequnece. Instead of providing just the predicate between `[SEP]` tokens, you're going to provide a context window for the predicate. 

This context helps the model disambiguate between the different word forms identical to the predicate word forms. The size of the context window is 3 words, meaning that alongside the predicate, its preceding and following word and put between the `[SEP]` tokens. For example, for the first occurence of 'saw', you'll create the following input sequence:

```
['[CLS]', 'she', 'saw', 'a', 'man', 'saw', 'a', 'tree', '[SEP]', 'she', 'saw', 'a' '[SEP]']
```

For the second occurence of 'saw', you'll create the following input sequence:

```
['[CLS]', 'she', 'saw', 'a', 'man', 'saw', 'a', 'tree', '[SEP]', 'man', 'saw', 'a' '[SEP]']
```

Using this context window, your model can learn to disambiguate between both occurences, reducing noise in the fine-tuning and (hopefully) improving the model's performance!

In [26]:
args = define_args(mode='advanced')

NameError: name 'define_args' is not defined

In [ ]:
main(args,mode='advanced', data_range=100)

## Limitations in the advanced model

Although the advanced model's representation addresses the issue of predicate ambiguity, it does not provide additional information about the tokens and their probable semantic role. Such representations do not entail explicit feature-based information depicting the relations between the predicate and the tokens, thus token classification is dependent on the prertained encodings of the model.

## Results

The results show that the advanced model did not actually manage to improve the performance of the basic model. Both models seem to perform similarly on the more frequent argument labels like ARG0, ARG1, and ARGM-MOD, showing that adding the addition of the context window does not itself suffice to significantly change how the model handles even the most frequent labels. Also, although the model shows slight improvements in some labels (e.g., ARG2's recall increases from 0.61 to 0.65), at the same time it also negatively impacts the performance of others (e.g., ARGM-ADV's precision decreases from 0.67 to 0.62). Overall, both models’ low performance could be partially attributed to the fact that the latter were trained on only one epoch, depriving them from the chance to learn more nuanced insights and to a further extend benefit from the addition of extra contextual information surrounding the predicate.

## Conclusion and future research

The results show that the baseline model, taking just the predicate, performs better than the advanced model. It is likely that the context window is not meaningful enough for the model to predict the correct arguments for semantic role labeling.

It is important to consider that the labels are assigned in terms of the subtokens fed to the transformer model, rather than the token itself, which might significantly impact the results. Future experiments should consider counting the label assigned to the token to evaluate the model's performance.

Additionally, future research should focus on different ways to provide a meaningful representation of the predicate to the model, making it robust enough for the semantic role labeling task.


## Group Contribution:

##### Ariana Britez:
- functions to map the labels to number for model input: get_label_mapping, map_labels_to_numbers, map_labels_in_dataframe
- function to get the list of labels for model input: get_labels_from_map
- function to compute the metrics during training, evaluation and inference: compute_metrics, compute_evaluation_metrics_from_csv
- function to load the transformer model for fine-tuning: load_srl_model
- function to load the dataset in format that model can handle: load_dataset
- function to save the model predictions with gold labels for evaluation: write_predictions_to_csv
- writing markdown from importing the model section until evaluation of the baseline model
- conclusion and future research

##### Kris Stallenberg:
- Writing tutorial-style documentation for the full pipeline.
- Adding print statements throughout the notebook clarifying the data structures.
- Explain the limitations of the basic model and how the advanced model solves these.
- Refactoring functions in the utils.py module.

##### Farnaz Bani Fatemi:
- Helping in preprocessing of data and fine tuning baseline model and tokenizer.
- Function to load and represent data for baseline model input form and extract gold arguments: read_data_as_sentence.
- Function to tokenize input form and assign each token label to its subtokens: tokenize_and_align_labels.
- Cell in main.ipynb to locally save fine-tuned model.
- Adding some commands to run code in colab.
- Testing some codes to help fix debugs and issues for models.

##### Szabolcs Pal
- Helped in preprocessing the data for advanced model
- Limitations and explanation of advanced model
- Main function used for advanced model
- Attemped to do postprocessing on the subtokens of the model output

##### Christina Karavida
- Worked on the baseline model (run into issues in the evaluation stage)
- Also created a version of the main function for the advanced model       
- Explained the results for the two models